In [183]:
import pandas as pd
import spacy

from tqdm import tqdm

Models: https://spacy.io/models/en#en_core_web_md

Entities: https://spacy.io/api/annotation#section-named-entities

In [2]:
nlp = spacy.load('en_core_web_md')

In [179]:
NE = {
    "B-EVENT": "B-event",
    "I-EVENT": "I-event",
    "B-LOC": "B-geo",
    "I-LOC": "I-geo",
    "B-GPE": "B-geo",
    "I-GPE": "I-geo",
    "B-PRODUCT": "B-obj",
    "I-PRODUCT": "I-obj",
    "B-WORK_OF_ART": "B-obj",
    "I-WORK_OF_ART": "I-obj",
    "B-ORG": "B-org",
    "I-ORG": "I-org",
    "B-PERSON": "B-per",
    "I-PERSON": "I-per",
    "B-TIME": "B-time",
    "I-TIME": "I-time",
    "B-DATE": "B-time",
    "I-DATE": "O",
    "B-NORP": "B-gpe",
    "I-NORP": "I-gpe",
    "B-FAC": "O",
    "I-FAC": "O",
    "B-LAW": "O",
    "I-LAW": "O",
    "B-LANGUAGE": "O",
    "I-LANGUAGE": "O",
    "B-PERCENT": "O",
    "I-PERCENT": "O",
    "B-MONEY": "O",
    "I-MONEY": "O",
    "B-QUANTITY": "O",
    "I-QUANTITY": "O",
    "B-ORDINAL": "O",
    "I-ORDINAL": "O",
    "B-CARDINAL": "O",
    "I-CARDINAL": "O",
    "O": "O"
}

In [4]:
df = pd.read_csv("../datasets/TrainNER.csv", sep=';',encoding='cp1250', names=["Sentence","Word","Tag","Category"])

In [5]:
df.head(10)

,Sentence,Word,Tag,Category
0,Sentence: 1,President,NNP,B-per
1,NaN,Karzai,NNP,I-per
2,NaN,thanked,VBD,O
3,NaN,his,PRP$,O
4,NaN,allies,NNS,O
5,NaN,for,IN,O
6,NaN,their,PRP$,O
7,NaN,help,NN,O
8,NaN,in,IN,O
9,NaN,battling,VBG,O


In [6]:
df['OriginalPrediction'] = "O"

In [27]:
indexes = df[df['Sentence'].isnull() == False].index.values.tolist()
indexes.append(df.shape[0]-1)
print(len(indexes))

9001


In [28]:
for i in tqdm(range(len(indexes)-1)):
    start = indexes[i]
    end = indexes[i+1]
    init_tokens = df[(df.index >= start) & (df.index < end)]['Word'].values.tolist()
    s = ' '.join(init_tokens)
    doc = nlp(s)
    ents = []
    for ent in doc.ents:
        l = ent.text.split(' ')
        if len(l) == 1:
            ents.append([ent.text, 'B-' + ent.label_])
        else:
            ents.append([l[0], 'B-' + ent.label_])
            for i in range(1,len(l)):
                ents.append([l[i], 'I-' + ent.label_])
    i = 0
    for j in range(len(init_tokens)):
        if i < len(ents) and init_tokens[j] == ents[i][0]:
            df.loc[df.index == start+j, 'OriginalPrediction'] = ents[i][1]
            i += 1
    df[(df.index >= start) & (df.index < end)]



  0%|          | 0/5 [00:00<?, ?it/s]

 20%|██        | 1/5 [00:00<00:00,  6.97it/s]

 60%|██████    | 3/5 [00:00<00:00,  6.79it/s]

100%|██████████| 5/5 [00:00<00:00,  8.15it/s]



In [30]:
df.tail()

,Sentence,Word,Tag,Category,OriginalPrediction
196641,NaN,and,CC,O,O
196642,NaN,foreign,JJ,O,O
196643,NaN,investment,NN,O,O
196644,NaN,.,.,O,O
196645,NaN,NaN,NaN,NaN,O


In [77]:
df.to_csv("../datasets/TrainNER_Pred", sep=';', encoding='utf-8')

In [180]:
df['Prediction'] = df.apply (lambda row: NE[row['OriginalPrediction']],axis=1)

In [181]:
results = {
    "Size": df.shape[0],
    "CorrectPredictionsWithO": df.loc[df['Category'] == df['Prediction']].shape[0],
    "CorrectEntity": df.loc[(df['Category'] != "O") & (df['Category'] == df['Prediction'])].shape[0],
    "AllEntities": df.loc[df['Category'] != "O"].shape[0],
    "AllPredictions": df.loc[df['Prediction'] != "O"].shape[0]
}
results

{'Size': 196646,
 'CorrectPredictionsWithO': 180117,
 'CorrectEntity': 17973,
 'AllEntities': 30036,
 'AllPredictions': 29225}

In [182]:
print("Absoulte accuracy: {0:.2f}%".format(results["CorrectPredictionsWithO"] / results["Size"] * 100))
print("Entity based accuracy: {0:.2f}%".format(results["CorrectEntity"] / results["AllEntities"] * 100))
print("Predicted entity based accuracy: {0:.2f}%".format(results["CorrectEntity"] / results["AllPredictions"] * 100))

Absoulte accuracy: 91.59%
Entity based accuracy: 59.84%
Predicted entity based accuracy: 61.50%


In [113]:
results["CorrectPredictionsWithO"] / results["Size"]

0.8725323678081426

In [76]:
df.loc[(df['Prediction'] != "O") & (df['Category'] != "O") & (df['Category'] != df['Prediction'])]

,Sentence,Word,Tag,Category,OriginalPrediction,Prediction,SentenceID
1,NaN,Karzai,NNP,I-per,B-GPE,B-gpe,0
48,NaN,Afghanistan,NNP,I-geo,B-GPE,B-gpe,2
87,Sentence: 5,Burma,NNP,B-geo,B-GPE,B-gpe,4
96,NaN,Khin,NNP,I-per,B-PERSON,B-per,4
131,NaN,Thura,NNP,I-org,B-PERSON,B-per,5
132,NaN,Shwe,NNP,I-org,I-PERSON,I-per,5
133,NaN,Mahn,NNP,I-org,I-PERSON,I-per,5
155,Sentence: 7,Thura,NNP,B-org,B-PERSON,B-per,6
156,NaN,Shwe,NNP,I-org,I-PERSON,I-per,6
157,NaN,Mahn,NNP,I-org,I-PERSON,I-per,6


In [75]:
for i in tqdm(range(len(indexes)-1)):
    start = indexes[i]
    end = indexes[i+1]
    for index in range(start,end):
        df.loc[df.index == index, 'SentenceID'] = i






  0%|          | 0/9000 [00:00<?, ?it/s]




  0%|          | 1/9000 [00:00<17:38,  8.50it/s]




  0%|          | 2/9000 [00:00<21:46,  6.89it/s]




  0%|          | 3/9000 [00:00<25:37,  5.85it/s]




  0%|          | 4/9000 [00:00<22:38,  6.62it/s]




  0%|          | 5/9000 [00:00<25:25,  5.89it/s]




  0%|          | 6/9000 [00:01<29:44,  5.04it/s]




  0%|          | 7/9000 [00:01<28:04,  5.34it/s]




  0%|          | 8/9000 [00:01<25:33,  5.87it/s]




  0%|          | 9/9000 [00:01<22:47,  6.57it/s]




  0%|          | 10/9000 [00:01<21:15,  7.05it/s]




  0%|          | 11/9000 [00:01<20:27,  7.33it/s]




  0%|          | 12/9000 [00:01<22:11,  6.75it/s]




  0%|          | 13/9000 [00:02<22:31,  6.65it/s]




  0%|          | 14/9000 [00:02<21:15,  7.05it/s]




  0%|          | 15/9000 [00:02<25:18,  5.92it/s]




  0%|          | 16/9000 [00:02<26:39,  5.62it/s]




  0%|          | 17/9000 [00:02<25:21,  5.91it/s]




  0%|          | 18/9000 [00:03<26:17,  5

  3%|▎         | 271/9000 [00:41<22:57,  6.34it/s]




  3%|▎         | 272/9000 [00:42<23:18,  6.24it/s]




  3%|▎         | 273/9000 [00:42<21:34,  6.74it/s]




  3%|▎         | 274/9000 [00:42<20:01,  7.26it/s]




  3%|▎         | 275/9000 [00:42<20:27,  7.11it/s]




  3%|▎         | 276/9000 [00:42<19:09,  7.59it/s]




  3%|▎         | 277/9000 [00:42<21:18,  6.82it/s]




  3%|▎         | 278/9000 [00:42<22:42,  6.40it/s]




  3%|▎         | 279/9000 [00:43<22:39,  6.41it/s]




  3%|▎         | 280/9000 [00:43<22:26,  6.48it/s]




  3%|▎         | 281/9000 [00:43<22:32,  6.45it/s]




  3%|▎         | 282/9000 [00:43<22:32,  6.45it/s]




  3%|▎         | 284/9000 [00:43<21:13,  6.85it/s]




  3%|▎         | 285/9000 [00:43<21:21,  6.80it/s]




  3%|▎         | 286/9000 [00:44<20:06,  7.22it/s]




  3%|▎         | 288/9000 [00:44<19:19,  7.51it/s]




  3%|▎         | 289/9000 [00:44<24:32,  5.91it/s]




  3%|▎         | 290/9000 [00:44<25:39,  5.66it/s]




  3%|▎    

  6%|▌         | 537/9000 [01:21<22:41,  6.21it/s]




  6%|▌         | 538/9000 [01:22<22:36,  6.24it/s]




  6%|▌         | 539/9000 [01:22<22:47,  6.19it/s]




  6%|▌         | 540/9000 [01:22<24:10,  5.83it/s]




  6%|▌         | 541/9000 [01:22<24:53,  5.66it/s]




  6%|▌         | 542/9000 [01:22<23:18,  6.05it/s]




  6%|▌         | 544/9000 [01:22<20:52,  6.75it/s]




  6%|▌         | 545/9000 [01:23<20:31,  6.86it/s]




  6%|▌         | 546/9000 [01:23<23:54,  5.90it/s]




  6%|▌         | 547/9000 [01:23<26:25,  5.33it/s]




  6%|▌         | 548/9000 [01:23<28:01,  5.03it/s]




  6%|▌         | 549/9000 [01:23<24:17,  5.80it/s]




  6%|▌         | 550/9000 [01:24<23:00,  6.12it/s]




  6%|▌         | 551/9000 [01:24<24:16,  5.80it/s]




  6%|▌         | 552/9000 [01:24<22:00,  6.40it/s]




  6%|▌         | 553/9000 [01:24<21:13,  6.63it/s]




  6%|▌         | 554/9000 [01:24<20:54,  6.73it/s]




  6%|▌         | 555/9000 [01:24<20:40,  6.81it/s]




  6%|▌    

  9%|▉         | 807/9000 [02:06<26:57,  5.07it/s]




  9%|▉         | 808/9000 [02:06<29:20,  4.65it/s]




  9%|▉         | 809/9000 [02:06<26:07,  5.23it/s]




  9%|▉         | 810/9000 [02:06<26:27,  5.16it/s]




  9%|▉         | 811/9000 [02:06<24:12,  5.64it/s]




  9%|▉         | 812/9000 [02:06<22:36,  6.03it/s]




  9%|▉         | 813/9000 [02:07<21:09,  6.45it/s]




  9%|▉         | 815/9000 [02:07<19:02,  7.16it/s]




  9%|▉         | 816/9000 [02:07<20:07,  6.78it/s]




  9%|▉         | 817/9000 [02:07<23:28,  5.81it/s]




  9%|▉         | 818/9000 [02:07<23:05,  5.91it/s]




  9%|▉         | 819/9000 [02:07<21:33,  6.33it/s]




  9%|▉         | 820/9000 [02:08<23:26,  5.82it/s]




  9%|▉         | 822/9000 [02:08<22:22,  6.09it/s]




  9%|▉         | 824/9000 [02:08<20:44,  6.57it/s]




  9%|▉         | 825/9000 [02:08<19:32,  6.97it/s]




  9%|▉         | 826/9000 [02:08<18:17,  7.45it/s]




  9%|▉         | 827/9000 [02:08<17:01,  8.00it/s]




  9%|▉    

 12%|█▏        | 1070/9000 [02:47<22:06,  5.98it/s]




 12%|█▏        | 1071/9000 [02:47<20:40,  6.39it/s]




 12%|█▏        | 1073/9000 [02:47<20:19,  6.50it/s]




 12%|█▏        | 1074/9000 [02:48<21:12,  6.23it/s]




 12%|█▏        | 1075/9000 [02:48<21:52,  6.04it/s]




 12%|█▏        | 1076/9000 [02:48<21:55,  6.02it/s]




 12%|█▏        | 1077/9000 [02:48<21:32,  6.13it/s]




 12%|█▏        | 1078/9000 [02:48<19:46,  6.67it/s]




 12%|█▏        | 1079/9000 [02:48<20:57,  6.30it/s]




 12%|█▏        | 1080/9000 [02:49<20:43,  6.37it/s]




 12%|█▏        | 1081/9000 [02:49<23:19,  5.66it/s]




 12%|█▏        | 1082/9000 [02:49<21:25,  6.16it/s]




 12%|█▏        | 1083/9000 [02:49<20:29,  6.44it/s]




 12%|█▏        | 1084/9000 [02:49<21:51,  6.03it/s]




 12%|█▏        | 1085/9000 [02:50<25:18,  5.21it/s]




 12%|█▏        | 1086/9000 [02:50<26:29,  4.98it/s]




 12%|█▏        | 1087/9000 [02:50<24:11,  5.45it/s]




 12%|█▏        | 1088/9000 [02:50<22:33,  5.85it

...

 89%|████████▊ | 7979/9000 [28:44<04:45,  3.57it/s]




 89%|████████▊ | 7980/9000 [28:44<04:57,  3.42it/s]




 89%|████████▊ | 7981/9000 [28:44<04:50,  3.50it/s]




 89%|████████▊ | 7982/9000 [28:45<04:21,  3.89it/s]




 89%|████████▊ | 7983/9000 [28:45<03:59,  4.24it/s]




 89%|████████▊ | 7984/9000 [28:45<04:35,  3.68it/s]




 89%|████████▊ | 7985/9000 [28:46<05:01,  3.37it/s]




 89%|████████▊ | 7986/9000 [28:46<04:12,  4.02it/s]




 89%|████████▊ | 7987/9000 [28:46<04:00,  4.21it/s]




 89%|████████▉ | 7988/9000 [28:46<04:09,  4.06it/s]




 89%|████████▉ | 7989/9000 [28:47<05:13,  3.23it/s]




 89%|████████▉ | 7990/9000 [28:47<04:43,  3.56it/s]




 89%|████████▉ | 7991/9000 [28:47<04:28,  3.75it/s]




 89%|████████▉ | 7992/9000 [28:47<03:46,  4.45it/s]




 89%|████████▉ | 7993/9000 [28:48<04:18,  3.89it/s]




 89%|████████▉ | 7994/9000 [28:48<04:24,  3.80it/s]




 89%|████████▉ | 7995/9000 [28:48<04:14,  3.95it/s]




 89%|████████▉ | 7996/9000 [28:48<04:09,  4.02it

 91%|█████████▏| 8221/9000 [29:41<02:37,  4.94it/s]




 91%|█████████▏| 8222/9000 [29:41<02:39,  4.88it/s]




 91%|█████████▏| 8223/9000 [29:41<02:38,  4.91it/s]




 91%|█████████▏| 8224/9000 [29:41<02:51,  4.53it/s]




 91%|█████████▏| 8225/9000 [29:42<02:58,  4.34it/s]




 91%|█████████▏| 8226/9000 [29:42<03:22,  3.82it/s]




 91%|█████████▏| 8227/9000 [29:42<03:29,  3.69it/s]




 91%|█████████▏| 8228/9000 [29:43<03:17,  3.90it/s]




 91%|█████████▏| 8229/9000 [29:43<02:52,  4.46it/s]




 91%|█████████▏| 8230/9000 [29:43<03:43,  3.45it/s]




 91%|█████████▏| 8231/9000 [29:44<04:05,  3.13it/s]




 91%|█████████▏| 8232/9000 [29:44<03:43,  3.44it/s]




 91%|█████████▏| 8233/9000 [29:44<02:59,  4.27it/s]




 91%|█████████▏| 8234/9000 [29:44<03:01,  4.23it/s]




 92%|█████████▏| 8235/9000 [29:44<02:48,  4.55it/s]




 92%|█████████▏| 8236/9000 [29:45<02:56,  4.33it/s]




 92%|█████████▏| 8237/9000 [29:45<03:13,  3.94it/s]




 92%|█████████▏| 8238/9000 [29:45<03:22,  3.76it

 94%|█████████▍| 8459/9000 [30:40<02:19,  3.88it/s]




 94%|█████████▍| 8460/9000 [30:40<02:22,  3.78it/s]




 94%|█████████▍| 8461/9000 [30:41<02:57,  3.03it/s]




 94%|█████████▍| 8462/9000 [30:41<03:19,  2.70it/s]




 94%|█████████▍| 8463/9000 [30:42<03:46,  2.37it/s]




 94%|█████████▍| 8464/9000 [30:42<03:20,  2.68it/s]




 94%|█████████▍| 8465/9000 [30:42<03:25,  2.61it/s]




 94%|█████████▍| 8466/9000 [30:43<03:12,  2.77it/s]




 94%|█████████▍| 8467/9000 [30:43<03:23,  2.62it/s]




 94%|█████████▍| 8468/9000 [30:43<03:07,  2.84it/s]




 94%|█████████▍| 8469/9000 [30:44<03:19,  2.66it/s]




 94%|█████████▍| 8470/9000 [30:44<02:58,  2.97it/s]




 94%|█████████▍| 8471/9000 [30:44<02:49,  3.12it/s]




 94%|█████████▍| 8472/9000 [30:44<02:27,  3.58it/s]




 94%|█████████▍| 8473/9000 [30:45<02:20,  3.75it/s]




 94%|█████████▍| 8474/9000 [30:45<01:56,  4.50it/s]




 94%|█████████▍| 8475/9000 [30:45<02:06,  4.14it/s]




 94%|█████████▍| 8476/9000 [30:45<02:08,  4.08it

 97%|█████████▋| 8694/9000 [31:38<01:09,  4.38it/s]




 97%|█████████▋| 8695/9000 [31:38<01:01,  4.97it/s]




 97%|█████████▋| 8696/9000 [31:39<01:14,  4.06it/s]




 97%|█████████▋| 8697/9000 [31:39<01:13,  4.13it/s]




 97%|█████████▋| 8698/9000 [31:39<01:28,  3.41it/s]




 97%|█████████▋| 8699/9000 [31:40<01:20,  3.75it/s]




 97%|█████████▋| 8700/9000 [31:40<01:20,  3.72it/s]




 97%|█████████▋| 8701/9000 [31:40<01:18,  3.82it/s]




 97%|█████████▋| 8702/9000 [31:40<01:17,  3.84it/s]




 97%|█████████▋| 8704/9000 [31:41<01:05,  4.55it/s]




 97%|█████████▋| 8705/9000 [31:41<01:05,  4.50it/s]




 97%|█████████▋| 8706/9000 [31:41<00:59,  4.94it/s]




 97%|█████████▋| 8707/9000 [31:41<00:52,  5.55it/s]




 97%|█████████▋| 8708/9000 [31:41<01:04,  4.55it/s]




 97%|█████████▋| 8709/9000 [31:42<01:09,  4.19it/s]




 97%|█████████▋| 8710/9000 [31:42<00:59,  4.84it/s]




 97%|█████████▋| 8711/9000 [31:42<01:14,  3.90it/s]




 97%|█████████▋| 8712/9000 [31:42<01:15,  3.79it

 99%|█████████▉| 8927/9000 [32:44<00:22,  3.19it/s]




 99%|█████████▉| 8928/9000 [32:44<00:22,  3.21it/s]




 99%|█████████▉| 8929/9000 [32:45<00:24,  2.91it/s]




 99%|█████████▉| 8930/9000 [32:45<00:22,  3.12it/s]




 99%|█████████▉| 8931/9000 [32:45<00:17,  3.92it/s]




 99%|█████████▉| 8932/9000 [32:45<00:18,  3.59it/s]




 99%|█████████▉| 8933/9000 [32:45<00:15,  4.32it/s]




 99%|█████████▉| 8934/9000 [32:46<00:15,  4.40it/s]




 99%|█████████▉| 8935/9000 [32:46<00:15,  4.21it/s]




 99%|█████████▉| 8936/9000 [32:46<00:15,  4.02it/s]




 99%|█████████▉| 8937/9000 [32:46<00:16,  3.77it/s]




 99%|█████████▉| 8938/9000 [32:47<00:17,  3.51it/s]




 99%|█████████▉| 8939/9000 [32:47<00:17,  3.52it/s]




 99%|█████████▉| 8940/9000 [32:47<00:17,  3.43it/s]




 99%|█████████▉| 8941/9000 [32:48<00:17,  3.45it/s]




 99%|█████████▉| 8942/9000 [32:48<00:17,  3.24it/s]




 99%|█████████▉| 8943/9000 [32:48<00:15,  3.58it/s]




 99%|█████████▉| 8944/9000 [32:48<00:14,  3.87it

In [ ]:
df.loc[(df['Prediction'] != "O") & (df['Category'] != "O") & (df['Category'] != df['Prediction'])]

In [102]:
df.keys()

Index(['Sentence', 'Word', 'Tag', 'Category', 'OriginalPrediction',
       'Prediction', 'SentenceID'],
      dtype='object')

In [88]:
df['SentenceID'].unique()

array([   0,    1,    2, ..., 8997, 8998, 8999])

In [111]:
def show_sentence(id):
    start = indexes[id]
    end = indexes[id+1]
    init_tokens = df[(df.index >= start) & (df.index < end)]['Word'].values.tolist()
    s = ' '.join(init_tokens)
    print(s)
    return df[(df.index >= start) & (df.index < end)][['Word','Category','Prediction','OriginalPrediction']]

In [112]:
show_sentence(1)

The commander of NATO 's Afghan force , British General David Richards , said the unity of command the transfer brought will enhance the effectiveness of the overall operation .


,Word,Category,Prediction,OriginalPrediction
12,The,O,O,O
13,commander,O,O,O
14,of,O,O,O
15,NATO,B-org,B-org,B-ORG
16,'s,O,O,O
17,Afghan,B-gpe,O,B-NORP
18,force,O,O,O
19,",",O,O,O
20,British,B-org,O,B-NORP
21,General,I-org,O,O


In [116]:
ne_list = ["B-event","I-event","B-geo","I-geo","B-gpe","I-gpe","B-obj","I-obj","B-obj","I-obj","B-org","I-org",
           "B-per","I-per","B-time","I-time","B-time","I-time"]

In [141]:
for cat in ne_list:
    n = df.loc[(df['Category'] == cat) & (df['Category'] != df['Prediction'])].shape[0]
    print('{} {}'.format(cat,n))

B-event 68
I-event 28
B-geo 1490
I-geo 489
B-gpe 715
I-gpe 49
B-obj 94
I-obj 46
B-obj 94
I-obj 46
B-org 2225
I-org 1188
B-per 1942
I-per 1658
B-time 979
I-time 457
B-time 979
I-time 457


In [160]:
for cat in df.loc[(df['Category'] == 'B-org') & (df['Category'] != df['Prediction'])]['OriginalPrediction'].unique():
    n = df.loc[(df['Category'] == 'B-org') & (df['OriginalPrediction'] == cat) & (df['Category'] != df['Prediction'])].shape[0]
    print(cat,n)

B-NORP 173
O 327
B-PERSON 194
I-ORG 655
B-GPE 664
B-EVENT 14
B-FAC 18
I-NORP 2
I-LOC 12
B-PRODUCT 13
I-FAC 23
I-GPE 49
I-LAW 14
B-LOC 23
I-DATE 2
I-EVENT 12
I-PERSON 12
B-DATE 5
B-WORK_OF_ART 2
I-WORK_OF_ART 4
B-CARDINAL 4
B-LAW 1
B-ORDINAL 1
I-MONEY 1


In [161]:
df.loc[(df['Category'] == 'I-org') & (df['OriginalPrediction'] == 'I-ORG')].shape[0]

1780

In [151]:
df.loc[(df['Category'] == 'B-org') & (df['OriginalPrediction'] == 'I-ORG')].shape[0]

655

In [135]:
df.loc[(df['Category'] == 'B-gpe') & (df['OriginalPrediction'] == 'B-NORP')].shape[0]

2568

In [136]:
df.loc[(df['Category'] != 'B-gpe') & (df['OriginalPrediction'] == 'B-NORP')].shape[0]

868

In [120]:
df.loc[(df['Category'] == 'B-geo')].shape[0]

6837

In [122]:
df.loc[(df['Category'] == 'B-geo')]['OriginalPrediction'].unique()

array(['O', 'B-GPE', 'B-LOC', 'I-NORP', 'B-NORP', 'B-PRODUCT', 'B-ORG',
       'B-EVENT', 'I-GPE', 'I-LOC', 'I-ORG', 'B-PERSON', 'B-FAC',
       'B-WORK_OF_ART', 'B-DATE', 'I-EVENT', 'I-FAC', 'I-PERSON',
       'I-WORK_OF_ART', 'I-DATE', 'B-LAW', 'B-CARDINAL'], dtype=object)

In [123]:
df.loc[(df['Category'] == 'B-geo') & (df['OriginalPrediction'] == 'B-GPE')].shape[0]

5068

In [124]:
df.loc[(df['Category'] == 'B-gpe')].shape[0]

3283

In [127]:
df.loc[(df['OriginalPrediction'] == 'B-GPE')].shape[0]

6861

In [165]:
results_by_models = [
    {'Size': 196646,
     'CorrectPredictionsWithO': 178588,
     'CorrectEntity': 18380, 
     'AllEntities': 30036, 
     'AllPredictions': 32141},
    {'Size': 196646, 
     'CorrectPredictionsWithO': 179008, 
     'CorrectEntity': 18607, 
     'AllEntities': 30036, 
     'AllPredictions': 32087},
    {'Size': 196646, 
     'CorrectPredictionsWithO': 179013, 
     'CorrectEntity': 18723, 
     'AllEntities': 30036, 
     'AllPredictions': 32220}
]
models = ['sm','md','lg']

In [173]:
res_out = []
for i in range(len(models)):
    o = [models[i]]
    res = results_by_models[i]
    o.append(res["CorrectPredictionsWithO"] / res["Size"] * 100)
    o.append(res["CorrectEntity"] / res["AllEntities"] * 100)
    o.append(res["CorrectEntity"] / res["AllPredictions"] * 100)
    res_out.append(o)

In [174]:
pd.DataFrame(res_out, columns=['model','accuracy','coverage','precision'])

,model,accuracy,coverage,precision
0,sm,90.817001,61.193235,57.185526
1,md,91.030583,61.948995,57.989217
2,lg,91.033126,62.335198,58.109870
